# Technical Factor - Williams %R (WR)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        (m_max(high, 14) - close) / (m_max(high, 14) - m_min(low, 14)) AS WR,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(WR < 0.5 AND m_lag(WR, 1) >= 0.5, 1, 0) AS TRBY1,
        IF(WR > 0.5 AND m_lag(WR, 1) <= 0.5, 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(WR < m_lag(WR, 1) AND m_lag(WR, 1) <= m_lag(WR, 2), 1, 0) AS MTBY1,
        IF(WR > m_lag(WR, 1) AND m_lag(WR, 1) >= m_lag(WR, 2), 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(WR > 0.8 AND (WR - m_lag(WR, 1)) < 0, 1, 0) AS RVBY1,
        IF(WR < 0.2 AND (WR - m_lag(WR, 1)) > 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(WR, 1) >= 0.8 AND WR < 0.8, 1, 0) AS BKBY1,
        IF(m_lag(WR, 1) <= 0.2 AND WR > 0.2, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        WR,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,WR,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-01-21,600519.SH,0.549766,0,0,0,0,0,0,0,0
1,2025-01-22,600519.SH,0.739792,0,0,0,0,0,0,0,0
2,2025-01-23,600519.SH,0.690322,0,0,0,0,0,0,0,0
3,2025-01-24,600519.SH,0.793597,0,0,0,0,0,0,0,0
4,2025-01-27,600519.SH,0.808498,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
225,2025-12-25,600519.SH,0.089305,0,0,0,0,0,0,0,0
226,2025-12-26,600519.SH,0.089977,0,0,0,0,0,1,0,0
227,2025-12-29,600519.SH,0.293982,0,0,0,1,0,0,0,1
228,2025-12-30,600519.SH,0.718894,0,1,0,1,0,0,0,0
